In [ ]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
monkey.patch_all(thread=False)
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = Trueee
server.serve_forever()
#http://127.0.0.2:5000/

127.0.0.1 - - [2020-07-04 00:01:44] "GET /bonds HTTP/1.1" 200 8073 0.031083
127.0.0.1 - - [2020-07-04 00:01:44] "GET /static/vendor/jquery/popper.min.js.map HTTP/1.1" 404 341 0.001004
127.0.0.1 - - [2020-07-04 00:01:44] "GET /static/vendor/bootstrsap4/bootstrap.min.js.map HTTP/1.1" 404 341 0.001000
127.0.0.1 - - [2020-07-04 00:01:44] "GET /sm/3a884fe0bdb97cb3a94b410e67cf38fdc248890d5581232077b3e6241e25cd21.map HTTP/1.1" 404 341 0.001004
127.0.0.1 - - [2020-07-04 00:01:45] "POST /query_data?table=smartlabbondsrus HTTP/1.1" 200 384486 0.060140
127.0.0.1 - - [2020-07-04 00:01:45] "POST /query_data?table=smartlabbondsusd HTTP/1.1" 200 26075 0.013040
127.0.0.1 - - [2020-07-04 00:01:45] "GET /static/favicon-16x16.png HTTP/1.1" 200 663 0.002007
127.0.0.1 - - [2020-07-04 00:04:03] "GET /bonds HTTP/1.1" 200 8073 0.009023
127.0.0.1 - - [2020-07-04 00:04:03] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57492 0.003035
127.0.0.1 - - [2020-07-04 00:04:03] "GET /static/css/templ

In [ ]:
import mygit as mg
from pathlib import Path
import os
folder=os.path.abspath('{}/..'.format(os.getcwd()))
#folder=Path(folder).parent
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
#gt.pyignore()
#git reset --hard origin/master gt.pull()
gt.push_commit(git_url,commit_msg='update')

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
%%time
from sys import path
import os
path.append(os.path.join(os.getcwd(),'src'))

from commons import convert_file_size,init_db_manager
from jobs import *
#a=job_world_fond_indexes(init_db_manager())

In [ ]:
from datetime import datetime
datetime.now().year

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

from external_sources import mosex,tradingview
import pandas as pd
a=mosex()
a=tradingview()

In [ ]:

import lxml.html as lh
from collections import Counter

class smartlab(request_session):
    __slots__='sources','link'
    def __init__(self):
        self.link:str=r'https://smart-lab.ru'
        self.sources:dict={'bonds':{'ОФЗ':{'link':r'/q/ofz/','currency':'RUR','paging':None},
                               'Муниципальные':{'link':r'/q/subfed/','currency':'RUR','paging':None},
                               'Корпоративные':{'link':r'/q/bonds/','currency':'RUR'
                                                ,r'paging':'order_by_val_to_day/desc/page{}/'},
                               'Еврооблигации':{'link':r'/q/eurobonds/','currency':'USD','paging':None}
                              },
                           'bonds_query':r'/q/bonds/'
                           
                     }
        self.limit_pages=100

    def __table_from_html(self,url:str,extra_fields_dict:dict=None)->tuple:
        mask="Unnamed: {}"
        """
        extra_fields_dict={field_text_val:[{'add_column_name':0,'xpath_text':"a/@href",'insert_position':0}]}
        """
        with self._init_session() as s:
            page=s.get(url)
        doc = lh.fromstring(page.content)
        tr_elements = doc.xpath('//tr')
        header=[h.text_content() for h in tr_elements[0]]
        ununique_columns=[ key for key,value in Counter(header).items() if value>1]
        if len(ununique_columns)>0:
            for i in range(len(header)):
                if header[i] in ununique_columns:
                    header[i]=mask.format(i)
                
        if extra_fields_dict:
            keys= list(extra_fields_dict.keys())
            for key in keys:
                value=extra_fields_dict[key]

                extra_fields_dict[header.index(key)]=extra_fields_dict.pop(key)
                for condition in value:
                    header.insert(condition['insert_position'],condition['add_column_name'])

        res=[]

        for row in tr_elements[1:]:           
            arr=[el.text_content() for el in row]
            if len(arr)>1:
                if extra_fields_dict:
                    for key,value in extra_fields_dict.items():

                        xml_element=row[key]
                        for condition in value:
                            arr.insert(condition['insert_position'],xml_element.xpath(condition['xpath_text']))

                res.append(arr)
        return header,res        
            
    def __extract_pages(self,url,paging,extra_fields_dict):
        i = 1
        res=[]
        while i < self.limit_pages:      
            page=self.__table_from_html(url+paging.format(i),extra_fields_dict={**extra_fields_dict})
            _table=page[1]
            if len(_table)==0:
                break
            else:
                _header=page[0]
                res+=_table
            i+=1  
        return _header,res
        
        
    def bonds_info(self)->dict:
        bonds_dict=self.sources['bonds']
        res={}
        for bond_group in list(bonds_dict.keys()):
            info=bonds_dict[bond_group]
            url,currency,paging=self.link+info['link'],info['currency'],info['paging']
            ticker='Тикер'
            extra_fields_dict={'Имя':[{'add_column_name':ticker
                                       ,'xpath_text':"a/@href"
                                       ,'insert_position':1
                                      } 
                                     ]}  
            if paging is None:
                res[bond_group]=(*self.__table_from_html(url,extra_fields_dict=extra_fields_dict),currency)
            else:
                 res[bond_group]=(*self.__extract_pages(url,paging,extra_fields_dict),currency)

        return res
    
a=smartlab().bonds_info()